In [4]:
from mrcnn.visualize import display_instances
from mrcnn import visualize

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import matplotlib.pyplot as plt
from mrcnn.visualize import * 

from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import numpy as np
import matplotlib.pyplot as plt

import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow.compat.v1 as tf
import tensorflow as tf
import keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM
import keras.callbacks

from PIL import Image, ImageDraw
from PIL import Image
import skimage 
import pandas as pd
import numpy as np

from mrcnn import utils

tf.compat.v1.disable_eager_execution()

In [11]:
class TestConfig(Config):
    NAME = "test"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 13 + 1
    BATCH_SIZE = 1
    
model = MaskRCNN(mode = 'inference',config = TestConfig(), model_dir= '')

model.load_weights('mask_rcnn_deepfashion2_0100.h5', by_name= True)

In [31]:
num = 1
img_dir = 'MuSinSa_photo_2/' + str(30000) + '.jpg'
img_1 = skimage.io.imread(img_dir)

A = model.detect([img_1]) 

In [32]:
A

[{'rois': array([[ 89,  48, 475, 470]]),
  'class_ids': array([1]),
  'scores': array([0.99948895], dtype=float32),
  'masks': array([[[False],
          [False],
          [False],
          ...,
          [False],
          [False],
          [False]],
  
         [[False],
          [False],
          [False],
          ...,
          [False],
          [False],
          [False]],
  
         [[False],
          [False],
          [False],
          ...,
          [False],
          [False],
          [False]],
  
         ...,
  
         [[False],
          [False],
          [False],
          ...,
          [False],
          [False],
          [False]],
  
         [[False],
          [False],
          [False],
          ...,
          [False],
          [False],
          [False]],
  
         [[False],
          [False],
          [False],
          ...,
          [False],
          [False],
          [False]]])}]

In [13]:
df_list_normal = pd.read_excel('MuSinSa_photo/Total_page_info.xlsx',index_col= 0)

lst_total = df_list_normal['index'].values

len(lst_total), lst_total

(6973, array([   0,    1,    2, ..., 6975, 6976, 6977], dtype=int64))

In [14]:
df_musina = pd.read_excel('First_to_Second_page.xlsx',index_col = 0).reset_index()

df_musina['index'] = [i for i in range(df_musina.shape[0])]

df_musina_2 = df_musina.iloc[df_list_normal['index'].values,:].copy()

df_musina_2.shape,df_musina_2.index

((6973, 7),
 Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             6968, 6969, 6970, 6971, 6972, 6973, 6974, 6975, 6976, 6977],
            dtype='int64', length=6973))

In [15]:
class_lst =[
    "short sleeve top",
    "long sleeve top",
    "short sleeve outwear",
    "long sleeve outwear",
    "vest",
    "sling",
    "shorts",
    "trousers",
    "skirt",
    "short sleeve dress",
    "long sleeve dress",
    "vest dress",
    "sling dress",
  ]

In [16]:
import urllib.request
import tqdm

def make_image_bouding_file(name, style, style_part, comment, tag, domain_url):
    
    lst_small = []
    img_dir = 'MuSinSa_photo/Origin/' + str(name) + '.jpg'
    
    img_1 = skimage.io.imread(img_dir)
    
    try:
        A = model.detect([img_1])    

        # 일단 그림은 탐지되는 경우에만 Applied 폴더에 넣기로 함!
        if len(A[0]['rois']) == 0:
            print('인식되지 않습니다.')
            class_name = 'Not_Detected'
            lst_small.append([name, 0,0,style,style_part,
                                 comment, tag, 0,domain_url])
        
        else:       
            lst_small.append([name, A[0]['class_ids'].tolist(),A[0]['scores'].tolist(),style,style_part,
                             comment, tag, A[0]['rois'].tolist(),domain_url])
            
        return lst_small
    
    except:
        class_name = 'Not_Detected'
        print('오류 발생')
        
        lst_small.append([name, 0,0,style,style_part,
                             comment, tag, 0,domain_url])
        
        return lst_small


    
    
    
    
    
    

In [17]:
lst_total_boundary = []
for idx_rem, (num, style, style_part, comment, tag, domain_url) in enumerate(zip(tqdm.tqdm(lst_total),
                                                                              df_musina_2['style'],df_musina_2['style_part'],
                                                                              df_musina_2['comment'],df_musina_2['tag'],
                                                                              df_musina_2['domain_url'])):
    
    lst_0 = make_image_bouding_file(str(num),style, style_part, comment, tag, domain_url)
    lst_total_boundary.append(lst_0)
    
    
lst_total_boundary_2 = lst_total_boundary

df_test = pd.DataFrame(lst_total_boundary_2)
df_test.columns = ['uniq_index','class_id','prob','style','style_part','comment',
                   'tag','coordinate','domain_url']


df_test.to_excel('Test.xlsx',encoding ='utf-8-sig')

  0%|                                                                               | 6/6973 [00:13<4:13:23,  2.18s/it]

인식되지 않습니다.


  0%|                                                                               | 7/6973 [00:15<4:12:50,  2.18s/it]

인식되지 않습니다.


  0%|                                                                               | 9/6973 [00:19<4:14:10,  2.19s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▉| 6968/6973 [4:17:41<00:11,  2.24s/it]

인식되지 않습니다.


100%|████████████████████████████████████████████████████████████████████████████| 6973/6973 [4:17:52<00:00,  2.22s/it]

인식되지 않습니다.


ValueError: Length mismatch: Expected axis has 1 elements, new values have 9 elements

In [19]:
# df_test.to_csv('임시.csv',encoding = 'utf-8-sig')

In [27]:
df_final = pd.DataFrame(np.array(lst_total_boundary_2).reshape(-1,9))
df_final.columns = ['uniq_index','class_id','prob','style','style_part','comment',
                   'tag','coordinate','domain_url']

In [28]:
# df_final.to_excel('Final.xlsx',encoding ='utf-8-sig')